# Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import configparser
import openai
import os

config = configparser.ConfigParser()
config.read('../pruebas_openai/config.ini')

OPENAI_MODEL = config['DEFAULT']['OPENAI_MODEL']
openai.api_key = config['DEFAULT']['OPENAI_API_KEY']
OPENAI_API_KEY = config['DEFAULT']['OPENAI_API_KEY']
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
from langchain.chat_models import ChatOpenAI
import requests
from IPython.display import Markdown
import pandas as pd

# Artículos

In [4]:
data = pd.read_csv("../data/contenido_2023_04_05_matriz_not_julia.csv")
data = data.iloc[[23]]

In [5]:
# Texto de ejemplo
texto_noticia = data['contenido'].values[0]
texto_noticia

'Ni la pandemia ni el cáncer de mama del que le operaron hace ahora un año han parado a Mabel Lozano en su lucha contra la prostitución y la trata, temas sobre los que tiene mucho que decir. Ganadora de un Goya al mejor cortometraje documental por \'Biografía del cadáver de una mujer\', ahora está metida en dos proyectos, un corto y un largo, escribiendo un libro sobre esa enfermedad y leyendo guiones que le han ofrecido como directora.\n\nMabel Lozano: "La prostitución sigue funcionando, porque los clientes encuentran triquiñuelas para ir a los pisos"\n\nHablamos hace un año, en pleno confinamiento, y me dijiste que la prostitución seguía funcionando incluso en esos momentos. ¿Qué consecuencias ha tenido en ella la pandemia?\n\nHa sido terrible. Las prostitutas son las más vulnerables: algunas han llegado a pasar mucha hambre, otras han aumentado su deuda y no han tenido ni 10 dólares para mandar a sus países de origen... y en numerosos casos hemos visto que la prostitución ha migrado

# Define Agents

In [6]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

2025-04-08 13:41:40.282909: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744112500.299275   26502 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744112500.304400   26502 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-08 13:41:40.320843: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
llm_openai = True

In [8]:
# Usando OpenAI
if llm_openai:
    llm = ChatOpenAI(model_name=OPENAI_MODEL,
                    api_key= OPENAI_API_KEY,
                    temperature=0.2)

# Usando Salamandra-2B
else:
    model_name = "BSC-LT/salamandra-2b"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    text_generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
    )

    llm = HuggingFacePipeline(pipeline=text_generator)

/tmp/ipykernel_26502/479705634.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name=OPENAI_MODEL,


In [9]:
# ----------------------
# 1. Agente Perfilador
# ----------------------
def crear_perfilador(profesion, genero, politica):
    pronombre = "ella" if genero == "mujer" else "él"
    articulo = "una" if genero == "mujer" else "un"
    role = f"Perfilador ({profesion}, {genero}, {politica})"

    goal = (
        "Describir el perfil de cada agente involucrado en el análisis del texto, "
        "incluyendo su profesión, género e ideología política."
    )

    backstory = (
        f"{articulo.capitalize()} {profesion} con identidad de género {genero} "
        f"y orientación política {politica}. Su único rol es identificar y comunicar el perfil "
        "del agente, sin realizar interpretaciones adicionales."
    )

    return Agent(
        role=role,
        goal=goal,
        backstory=backstory,
        verbose=True,
        llm=llm
    )

In [10]:
# ----------------------
# 2. Agente Analista
# ----------------------
def crear_analista():
    role = "Analista"

    goal = "Detectar estereotipos, sesgos de género o lenguaje sexista en textos periodísticos."
    backstory = "Su labor es identificar sesgos implícitos o explícitos en el texto."

    return Agent(
        role=role,
        goal=goal,
        backstory=backstory,
        verbose=True,
        llm=llm
    )

In [11]:
# ----------------------
# 3. Agente Redactor
# ----------------------
def crear_redactor():
    role = "Redactor"
    goal = "Reescribir el texto original usando lenguaje inclusivo y eliminando estereotipos."
    backstory = "Se especializa en transformar lenguaje sesgado en narrativas igualitarias."

    return Agent(
        role=role,
        goal=goal,
        backstory=backstory,
        verbose=True,
        llm=llm
    )

In [12]:
# ----------------------
# 4. Agente Estilizador
# ----------------------
def crear_estilizador():
    role = "Estilizador"
    goal = "Convertir los resultados anteriores en un informe estructurado en Markdown."
    backstory = "Asistente especializado en presentación clara de resultados. No realiza análisis, sino que organiza y mejora la visualización del contenido."

    return Agent(
        role=role,
        goal=goal,
        backstory=backstory,
        verbose=True,
        llm=llm
    )

## Crear agentes

In [13]:
# Define los parámetros para el agente perfilador
profesion = "obrero" # "jueza", "abogada", "periodista", "investigadora", "profesora", "médica", "psicóloga", "trabajadora social"
genero = "hombre" # "mujer", "hombre"
politica = "conservadora" # "progresista", "conservadora", "neutra"

In [14]:
# Crear agentes
agentes = {}

# Crear tipos de agentes
agentes["perfilador"] = crear_perfilador(profesion, genero, politica)
agentes["analista"] = crear_analista()
agentes["redactor"] = crear_redactor()
agentes["estilizador"] = crear_estilizador()


## Crear tareas

In [15]:
tareas = [
    Task(
        description=(
            "Tu tarea es describir el perfil del agente que ha realizado el análisis. "
            "Solo debes identificar y comunicar las siguientes características del agente:\n\n"
            "- Profesión del agente\n"
            "- Género del agente\n"
            "- Ideología política del agente\n\n"
            "No debes hacer ningún análisis sobre el texto ni sobre el contenido de los análisis realizados."
        ),
        expected_output=(
            "Perfil del agente que ha realizado el análisis:\n"
            f"Profesión: {profesion}\n"
            f"Género: {genero}\n"
            f"Ideología política: {politica}"
        ),
        agent=agentes["perfilador"]
    ),
    Task(
    description=(
            f"""Analiza el siguiente texto completo con atención al contexto y la intención del autor.

    Tu objetivo es detectar frases que perpetúen estereotipos, lenguaje sexista o sesgos de género de forma explícita o implícita.  
    ⚠️ Ten en cuenta que algunas frases podrían parecer problemáticas en sí mismas, pero no lo son si el texto las está usando para **criticar** esos sesgos.  
    No marques como sesgada ninguna frase si forma parte de una **denuncia o crítica** a ese mismo lenguaje.

    1. Enumera las frases que verdaderamente refuercen o reproduzcan estereotipos o sesgos de género. Justifica cada una.
    2. En el texto original:
        - Usa `<span style="color: red; font-weight: bold;">... </span>` para marcar frases con sesgos o estereotipos.

    Texto original:
    {texto_noticia}
    """
        ),
        expected_output=(
            "Frases problemáticas con justificación + texto original anotado con Markdown."
        ),
        agent=agentes["analista"]
    ),
    Task(
        description="Reescribe el texto con lenguaje inclusivo y sin estereotipos.",
        expected_output="Nueva versión del texto con lenguaje igualitario.",
        agent=agentes["redactor"],
        input="previous_task_output"
    ),
    Task(
        description="Toma todos los análisis anteriores y preséntalos en un informe en formato Markdown con títulos, listas, separadores y claridad visual.",
        expected_output="Informe en Markdown limpio y bien estructurado.",
        agent=agentes["estilizador"],
        input="previous_task_output"
    )
]

# Orchestrate Agents

In [16]:
crew = Crew(
    agents=list(agentes.values()),
    tasks=tareas,
    verbose=True
)

# Execute

In [17]:
resultado = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8e48eb0d-ac5e-4630-81a7-be3dc2086bc9                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 96d5fd14-9b65-4fb2-90d8-6701356d1bf2
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 96d5fd14-9b65-4fb2-90d8-6701356d1bf2
       Status: Executing Task...
    └── 🤖 Agent: Perfilador (obrero, hombre, conservadora)
            Status: In Progress

# Agent: Perfilador (obrero, hombre, conservadora)
## Task: Tu tarea es describir el perfil del agente que ha realizado el análisis. Solo debes identificar y comunicar las siguientes características del agente:

- Profesión del agente
- Género del agente
- Ideología política del agente

No debes hacer ningún análisis sobre el texto ni sobre el contenido de los análisis realizados.


🚀 Crew: crew
└── 📋 Task: 96d5fd14-9b65-4fb2-90d8-6701356d1bf2
       Status: Executing Task...
    └── 🤖 Agent: Perfilador (obrero, hombre, conservadora)
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Estilizador
## Final Answer: 
# Informe sobre la Prostitución y la Trata

## Perfil del Agente que Realiza el Análisis
- **Profesión:** Obrero  
- **Género:** Hombre  
- **Ideología Política:** Conservadora  

---

## Frases que Refuerzan o Reproducen Estereotipos o Sesgos de Género

1. **"Las prostitutas son las más vulnerables: algunas han llegado a pasar mucha hambre, otras han aumentado su deuda y no han tenido ni 10 dólares para mandar a sus países de origen..."**
   - **Justificación:** Esta frase perpetúa el estereotipo de que las mujeres en la prostitución son siempre víctimas y no menciona su agencia o capacidad de decisión. Además, refuerza la idea de que las mujeres en esta situación son inherentemente débiles y dependientes.

2. **"Nuestros chavales están 'educando' la sexualidad en la pornografía, lo que les hace creer que tienen derecho a comprar a un ser humano."**
   - **Justificación:** Aunque la frase critica la educación sexual deficiente, también implica 

🚀 Crew: crew
├── 📋 Task: 96d5fd14-9b65-4fb2-90d8-6701356d1bf2
│      Assigned to: Perfilador (obrero, hombre, conservadora)
│      Status: ✅ Completed
│   └── 🤖 Agent: Perfilador (obrero, hombre, conservadora)
│           Status: ✅ Completed
├── 📋 Task: 6d23f0a6-83cc-4de8-b39c-a7d077256932
│      Assigned to: Analista
│      Status: ✅ Completed
│   └── 🤖 Agent: Analista
│           Status: ✅ Completed
├── 📋 Task: d1ca8720-34ac-4784-9d66-bc1b07a2f02c
│      Assigned to: Redactor
│      Status: ✅ Completed
│   └── 🤖 Agent: Redactor
│           Status: ✅ Completed
└── 📋 Task: a2554cae-5a49-43a9-b461-2171fc91312a
       Status: Executing Task...
    └── 🤖 Agent: Estilizador
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 96d5fd14-9b65-4fb2-90d8-6701356d1bf2
│      Assigned to: Perfilador (obrero, hombre, conservadora)
│      Status: ✅ Completed
│   └── 🤖 Agent: Perfilador (obrero, hombre, conservadora)
│           Status: ✅ Completed
├── 📋 Task: 6d23f0a6-83cc-4de8-b39c-a7d077256932
│      Assigned to: Analista
│      Status: ✅ Completed
│   └── 🤖 Agent: Analista
│           Status: ✅ Completed
├── 📋 Task: d1ca8720-34ac-4784-9d66-bc1b07a2f02c
│      Assigned to: Redactor
│      Status: ✅ Completed
│   └── 🤖 Agent: Redactor
│           Status: ✅ Completed
└── 📋 Task: a2554cae-5a49-43a9-b461-2171fc91312a
       Assigned to: Estilizador
       Status: ✅ Completed
    └── 🤖 Agent: Estilizador
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a2554cae-5a49-43a9-b461-2171fc91312a                                                                     │
│  Agent: Estilizador                                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8e48eb0d-ac5e-4630-81a7-be3dc2086bc9                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [18]:
perfilador_output = resultado.tasks_output[0].raw
display(Markdown(perfilador_output))

Perfil del agente que ha realizado el análisis:  
Profesión: obrero  
Género: hombre  
Ideología política: conservadora

In [19]:
analista_output = resultado.tasks_output[1].raw
display(Markdown(analista_output))

1. Frases que refuerzan o reproducen estereotipos o sesgos de género:

   - "Las prostitutas son las más vulnerables: algunas han llegado a pasar mucha hambre, otras han aumentado su deuda y no han tenido ni 10 dólares para mandar a sus países de origen..."
     - Justificación: Esta frase perpetúa el estereotipo de que las mujeres en la prostitución son siempre víctimas y no menciona su agencia o capacidad de decisión. Además, refuerza la idea de que las mujeres en esta situación son inherentemente débiles y dependientes.

   - "Nuestros chavales están 'educando' la sexualidad en la pornografía, lo que les hace creer que tienen derecho a comprar a un ser humano."
     - Justificación: Aunque la frase critica la educación sexual deficiente, también implica que los hombres jóvenes son inherentemente consumidores de mujeres, lo que refuerza un estereotipo de género que asocia a los hombres con la objetivación y compra de mujeres.

   - "Es algo que no debería tener cabida en este siglo."
     - Justificación: Esta frase puede interpretarse como una crítica a la prostitución, pero también puede implicar que la prostitución es un fenómeno que debería ser erradicado, lo que puede desestimar las experiencias y decisiones de las mujeres que eligen este camino, perpetuando la idea de que las mujeres no pueden tomar decisiones informadas sobre su propio cuerpo.

2. Texto original anotado con Markdown:

   Ni la pandemia ni el cáncer de mama del que le operaron hace ahora un año han parado a Mabel Lozano en su lucha contra la prostitución y la trata, temas sobre los que tiene mucho que decir. Ganadora de un Goya al mejor cortometraje documental por 'Biografía del cadáver de una mujer', ahora está metida en dos proyectos, un corto y un largo, escribiendo un libro sobre esa enfermedad y leyendo guiones que le han ofrecido como directora.

   Mabel Lozano: "La prostitución sigue funcionando, porque los clientes encuentran triquiñuelas para ir a los pisos"

   Hablamos hace un año, en pleno confinamiento, y me dijiste que la prostitución seguía funcionando incluso en esos momentos. ¿Qué consecuencias ha tenido en ella la pandemia?

   Ha sido terrible. <span style="color: red; font-weight: bold;">Las prostitutas son las más vulnerables: algunas han llegado a pasar mucha hambre, otras han aumentado su deuda y no han tenido ni 10 dólares para mandar a sus países de origen...</span> y en numerosos casos hemos visto que la prostitución ha migrado a las fronteras digitales. Muchas, sobre todo las jóvenes, han estado recibiendo a ese demandante de sexo de pago a través de las nuevas tecnologías.

   El Gobierno se mueve en torno a este tema.

   La ministra de Igualdad, Irene Montero, está preparando una ley que se va a presentar ya en el Congreso, la de Libertad sexual, la famosa del 'sí es sí', en la que se va a abordar el tema de las tercerías locativas, es decir, el proxenetismo consentido. En España las mujeres ejercen la prostitución por cuenta de un tercero, que es el que se lucra. Hay que ir hacia el lobby proxeneta para que nadie pueda lucrarse de la prostitución de otro. Lo que no aborda esta ley, y sería muy importante, es que las fuerzas y cuerpos de seguridad del Estado, la judicatura y sobre todo la fiscalía contaran con especialistas en este tema, igual que lo han hecho con la violencia de género.

   ¿Estás a favor de esta ley?

   Absolutamente a favor de que introduzca el tema de las tercerías locativas. Pero podría haber sido mucho más valiente y haber abordado la fiscalía especializada. En el ámbito de la violencia de género se ha dado un paso al frente porque se ha hecho esto.

   ¿Y las Comunidades Autónomas están haciendo algo?

   Estamos viendo iniciativas para hacer políticas abolicionistas, es decir, revestir de derechos a las mujeres, abordar el tema de la reinserción, de la educación sexual de los jóvenes... Eso está muy bien, pero es un parche, porque lo que necesitamos de verdad es una ley orgánica abolicionista. ¿Por qué no la hacemos?

   ¿Hay algún país que haya legislado y estés de acuerdo con cómo lo ha hecho?

   El último en legislar estas políticas abolicionistas es Francia, con una ley mucho más avanzada, que viste de derechos a las mujeres. Porque de lo que se trata es de dar alternativas y oportunidades. ¿Erradica la prostitución y la trata al cien por cien? No, y en muchos casos la esconde, pero hay menos explotación.

   ¿En qué cifras nos movemos en España de clientes de la prostitución? ¿Es el país con más clientes de Europa?

   Hay una encuesta que dice, y no lo digo yo, porque a todos los que les pregunto me dicen que no, que un 38% de los hombres españoles reconoce utilizarla, lo que corresponde a 8 millones y medio. Es el país de Europa que más consume y uno de los más del mundo, porque la prostitución está muy normalizada y legitimada.

   ¿A qué puede deberse esto?

   A que está enquistada desde siempre, de toda la vida, y se normaliza también por la falta de educación sexual. <span style="color: red; font-weight: bold;">No la hay. Nuestros chavales están 'educando' la sexualidad en la pornografía, lo que les hace creer que tienen derecho a comprar a un ser humano.</span> Nos hace falta muchísima educación en los entornos familiares y en los colegios. La Ley Celáa, de la que todo el mundo habla pestes, aborda la sexualidad como una asignatura, lo cual es magnífico.

   ¿Cómo se enfrentan los jóvenes a la prostitución, es decir, cómo viene el futuro?

   Viene lleno de pornografía, lo que llamo prostitución 2.0, la pornografía en vivo. Para escribir mi último libro, 'PornoXplotación', investigué durante casi cinco años junto a un inspector de la policía y vimos que la prostitución está migrando a esas fronteras digitales, y es a eso a lo que vamos a tener que enfrentarnos. La inteligencia artificial, que es magnífica y nos facilita la vida, también va a dejar muchas víctimas en los dos lados: por una parte niñas explotadas, secuestradas, engañadas, grooming, sexting..., y al otro lado de la pantalla niños cada vez más jóvenes que llegan a la pornografía sin un pensamiento crítico. Es un fenómeno que parece inocuo pero es peligrosísimo, turbio y en muchos casos a quienes la consumen desde muy temprana edad los vemos comprando mujeres en las rotondas a los 18, 19, 20 años... <span style="color: red; font-weight: bold;">Es algo que no debería tener cabida en este siglo.</span>

   ¿Cuál es tu solución a esta lacra?

   No es una lacra, sino un fenómeno delictivo, porque trata y prostitución van de la mano. Porque no se puede hablar de prostitución sin hablar de trata sexual, ni de esta sin la primera, y no se puede hablar de pornografía sin hablar de prostitución. Hay dos pilares muy importantes: la legislación -leyes orgánicas que penalicen este fenómeno delictivo, a los proxenetas y a los demandantes de sexo de pago como cómplices, y que den derechos a las mujeres- y la educación sexual de los jóvenes. A partir de ahí intentaremos poner un grano de arena todos, madres y padres, como educadores, porque si no lo dejamos todo en el tejado de los políticos. ¿Qué puedo hacer yo en mi casa? Soy educador. ¿Qué hago yo? Soy cineasta y utilizo el cine como herramienta de transformación social, acabo de ganar un Goya con 'Biografía del cadáver de una mujer' y esto ha supuesto un altavoz de gran impacto, veo cómo se empieza a mover ficha. Hasta ahora no ha habido una voluntad política ni en el PP ni en el PSOE, pero no es el momento de parchear, sino de hacer leyes con valentía.

In [20]:
redactor_output = resultado.tasks_output[2].raw
display(Markdown(redactor_output))

Perfil de la persona que ha realizado el análisis:  
Profesión: persona trabajadora  
Género: no especificado  
Ideología política: no especificada  

----------

1. Frases que refuerzan o reproducen estereotipos o sesgos de género:

   - "Las personas en situación de prostitución son las más vulnerables: algunas han llegado a pasar mucha hambre, otras han aumentado su deuda y no han tenido ni 10 dólares para mandar a sus países de origen..."
     - Justificación: Esta frase reconoce la vulnerabilidad de las personas en situación de prostitución sin desestimar su agencia o capacidad de decisión. Además, evita reforzar la idea de que estas personas son inherentemente débiles y dependientes.

   - "La juventud está 'educando' la sexualidad a través de la pornografía, lo que les hace creer que tienen derecho a comprar a un ser humano."
     - Justificación: Esta frase critica la educación sexual deficiente sin implicar que los jóvenes son inherentemente consumidores de personas, lo que ayuda a desestigmatizar la objetivación.

   - "Es algo que no debería tener cabida en este siglo."
     - Justificación: Esta frase puede interpretarse como una crítica a la prostitución, pero también puede implicar que la prostitución es un fenómeno que debería ser erradicado, lo que puede desestimar las experiencias y decisiones de las personas que eligen este camino, perpetuando la idea de que no pueden tomar decisiones informadas sobre su propio cuerpo.

2. Texto original anotado con Markdown:

   Ni la pandemia ni el cáncer de mama del que le operaron hace ahora un año han parado a Mabel Lozano en su lucha contra la prostitución y la trata, temas sobre los que tiene mucho que decir. Ganadora de un Goya al mejor cortometraje documental por 'Biografía del cadáver de una mujer', ahora está metida en dos proyectos, un corto y un largo, escribiendo un libro sobre esa enfermedad y leyendo guiones que le han ofrecido como directora.

   Mabel Lozano: "La prostitución sigue funcionando, porque las personas demandantes encuentran triquiñuelas para ir a los pisos."

   Hablamos hace un año, en pleno confinamiento, y me dijiste que la prostitución seguía funcionando incluso en esos momentos. ¿Qué consecuencias ha tenido en ella la pandemia?

   Ha sido terrible. <span style="color: red; font-weight: bold;">Las personas en situación de prostitución son las más vulnerables: algunas han llegado a pasar mucha hambre, otras han aumentado su deuda y no han tenido ni 10 dólares para mandar a sus países de origen...</span> y en numerosos casos hemos visto que la prostitución ha migrado a las fronteras digitales. Muchas, sobre todo las jóvenes, han estado recibiendo a esas personas demandantes de sexo de pago a través de las nuevas tecnologías.

   El Gobierno se mueve en torno a este tema.

   La ministra de Igualdad, Irene Montero, está preparando una ley que se va a presentar ya en el Congreso, la de Libertad sexual, la famosa del 'sí es sí', en la que se va a abordar el tema de las tercerías locativas, es decir, el proxenetismo consentido. En España, las personas ejercen la prostitución por cuenta de un tercero, que es quien se lucra. Hay que ir hacia el lobby proxeneta para que nadie pueda lucrarse de la prostitución de otra persona. Lo que no aborda esta ley, y sería muy importante, es que las fuerzas y cuerpos de seguridad del Estado, la judicatura y sobre todo la fiscalía cuenten con especialistas en este tema, igual que lo han hecho con la violencia de género.

   ¿Estás a favor de esta ley?

   Absolutamente a favor de que introduzca el tema de las tercerías locativas. Pero podría haber sido mucho más valiente y haber abordado la fiscalía especializada. En el ámbito de la violencia de género se ha dado un paso al frente porque se ha hecho esto.

   ¿Y las Comunidades Autónomas están haciendo algo?

   Estamos viendo iniciativas para hacer políticas abolicionistas, es decir, revestir de derechos a las personas, abordar el tema de la reinserción, de la educación sexual de la juventud... Eso está muy bien, pero es un parche, porque lo que necesitamos de verdad es una ley orgánica abolicionista. ¿Por qué no la hacemos?

   ¿Hay algún país que haya legislado y estés de acuerdo con cómo lo ha hecho?

   El último en legislar estas políticas abolicionistas es Francia, con una ley mucho más avanzada, que viste de derechos a las personas. Porque de lo que se trata es de dar alternativas y oportunidades. ¿Erradica la prostitución y la trata al cien por cien? No, y en muchos casos la esconde, pero hay menos explotación.

   ¿En qué cifras nos movemos en España de personas demandantes de la prostitución? ¿Es el país con más clientes de Europa?

   Hay una encuesta que dice, y no lo digo yo, porque a todas las personas que les pregunto me dicen que no, que un 38% de las personas españolas reconoce utilizarla, lo que corresponde a 8 millones y medio. Es el país de Europa que más consume y uno de los más del mundo, porque la prostitución está muy normalizada y legitimada.

   ¿A qué puede deberse esto?

   A que está enquistada desde siempre, de toda la vida, y se normaliza también por la falta de educación sexual. <span style="color: red; font-weight: bold;">No la hay. La juventud está 'educando' la sexualidad a través de la pornografía, lo que les hace creer que tienen derecho a comprar a un ser humano.</span> Nos hace falta muchísima educación en los entornos familiares y en los colegios. La Ley Celáa, de la que todo el mundo habla pestes, aborda la sexualidad como una asignatura, lo cual es magnífico.

   ¿Cómo se enfrentan las personas jóvenes a la prostitución, es decir, cómo viene el futuro?

   Viene lleno de pornografía, lo que llamo prostitución 2.0, la pornografía en vivo. Para escribir mi último libro, 'PornoXplotación', investigué durante casi cinco años junto a un inspector de la policía y vimos que la prostitución está migrando a esas fronteras digitales, y es a eso a lo que vamos a tener que enfrentarnos. La inteligencia artificial, que es magnífica y nos facilita la vida, también va a dejar muchas víctimas en los dos lados: por una parte, personas jóvenes explotadas, secuestradas, engañadas, grooming, sexting..., y al otro lado de la pantalla, personas cada vez más jóvenes que llegan a la pornografía sin un pensamiento crítico. Es un fenómeno que parece inocuo pero es peligrosísimo, turbio y en muchos casos a quienes la consumen desde muy temprana edad los vemos comprando personas en las rotondas a los 18, 19, 20 años... <span style="color: red; font-weight: bold;">Es algo que no debería tener cabida en este siglo.</span>

   ¿Cuál es tu solución a esta problemática?

   No es una problemática, sino un fenómeno delictivo, porque trata y prostitución van de la mano. Porque no se puede hablar de prostitución sin hablar de trata sexual, ni de esta sin la primera, y no se puede hablar de pornografía sin hablar de prostitución. Hay dos pilares muy importantes: la legislación -leyes orgánicas que penalicen este fenómeno delictivo, a los proxenetas y a las personas demandantes de sexo de pago como cómplices, y que den derechos a las personas- y la educación sexual de la juventud. A partir de ahí intentaremos poner un grano de arena todos, madres y padres, como educadores, porque si no lo dejamos todo en el tejado de las personas políticas. ¿Qué puedo hacer yo en mi casa? Soy educador. ¿Qué hago yo? Soy cineasta y utilizo el cine como herramienta de transformación social, acabo de ganar un Goya con 'Biografía del cadáver de una mujer' y esto ha supuesto un altavoz de gran impacto, veo cómo se empieza a mover ficha. Hasta ahora no ha habido una voluntad política ni en el PP ni en el PSOE, pero no es el momento de parchear, sino de hacer leyes con valentía.

In [21]:
estilizador_output = resultado.tasks_output[3].raw
display(Markdown(estilizador_output))

# Informe sobre la Prostitución y la Trata

## Perfil del Agente que Realiza el Análisis
- **Profesión:** Obrero  
- **Género:** Hombre  
- **Ideología Política:** Conservadora  

---

## Frases que Refuerzan o Reproducen Estereotipos o Sesgos de Género

1. **"Las prostitutas son las más vulnerables: algunas han llegado a pasar mucha hambre, otras han aumentado su deuda y no han tenido ni 10 dólares para mandar a sus países de origen..."**
   - **Justificación:** Esta frase perpetúa el estereotipo de que las mujeres en la prostitución son siempre víctimas y no menciona su agencia o capacidad de decisión. Además, refuerza la idea de que las mujeres en esta situación son inherentemente débiles y dependientes.

2. **"Nuestros chavales están 'educando' la sexualidad en la pornografía, lo que les hace creer que tienen derecho a comprar a un ser humano."**
   - **Justificación:** Aunque la frase critica la educación sexual deficiente, también implica que los hombres jóvenes son inherentemente consumidores de mujeres, lo que refuerza un estereotipo de género que asocia a los hombres con la objetivación y compra de mujeres.

3. **"Es algo que no debería tener cabida en este siglo."**
   - **Justificación:** Esta frase puede interpretarse como una crítica a la prostitución, pero también puede implicar que la prostitución es un fenómeno que debería ser erradicado, lo que puede desestimar las experiencias y decisiones de las mujeres que eligen este camino, perpetuando la idea de que las mujeres no pueden tomar decisiones informadas sobre su propio cuerpo.

---

## Texto Original Anotado con Markdown

Ni la pandemia ni el cáncer de mama del que le operaron hace ahora un año han parado a Mabel Lozano en su lucha contra la prostitución y la trata, temas sobre los que tiene mucho que decir. Ganadora de un Goya al mejor cortometraje documental por 'Biografía del cadáver de una mujer', ahora está metida en dos proyectos, un corto y un largo, escribiendo un libro sobre esa enfermedad y leyendo guiones que le han ofrecido como directora.

**Mabel Lozano:** "La prostitución sigue funcionando, porque los clientes encuentran triquiñuelas para ir a los pisos."

### Consecuencias de la Pandemia

Hablamos hace un año, en pleno confinamiento, y me dijiste que la prostitución seguía funcionando incluso en esos momentos. ¿Qué consecuencias ha tenido en ella la pandemia?

Ha sido terrible. **Las prostitutas son las más vulnerables: algunas han llegado a pasar mucha hambre, otras han aumentado su deuda y no han tenido ni 10 dólares para mandar a sus países de origen...** y en numerosos casos hemos visto que la prostitución ha migrado a las fronteras digitales. Muchas, sobre todo las jóvenes, han estado recibiendo a ese demandante de sexo de pago a través de las nuevas tecnologías.

### Movimiento del Gobierno

El Gobierno se mueve en torno a este tema.

La ministra de Igualdad, Irene Montero, está preparando una ley que se va a presentar ya en el Congreso, la de Libertad sexual, la famosa del 'sí es sí', en la que se va a abordar el tema de las tercerías locativas, es decir, el proxenetismo consentido. En España las mujeres ejercen la prostitución por cuenta de un tercero, que es el que se lucra. Hay que ir hacia el lobby proxeneta para que nadie pueda lucrarse de la prostitución de otro. Lo que no aborda esta ley, y sería muy importante, es que las fuerzas y cuerpos de seguridad del Estado, la judicatura y sobre todo la fiscalía contaran con especialistas en este tema, igual que lo han hecho con la violencia de género.

### Opinión sobre la Ley

¿Estás a favor de esta ley?

Absolutamente a favor de que introduzca el tema de las tercerías locativas. Pero podría haber sido mucho más valiente y haber abordado la fiscalía especializada. En el ámbito de la violencia de género se ha dado un paso al frente porque se ha hecho esto.

### Iniciativas de Comunidades Autónomas

¿Y las Comunidades Autónomas están haciendo algo?

Estamos viendo iniciativas para hacer políticas abolicionistas, es decir, revestir de derechos a las mujeres, abordar el tema de la reinserción, de la educación sexual de los jóvenes... Eso está muy bien, pero es un parche, porque lo que necesitamos de verdad es una ley orgánica abolicionista. ¿Por qué no la hacemos?

### Legislación en Otros Países

¿Hay algún país que haya legislado y estés de acuerdo con cómo lo ha hecho?

El último en legislar estas políticas abolicionistas es Francia, con una ley mucho más avanzada, que viste de derechos a las mujeres. Porque de lo que se trata es de dar alternativas y oportunidades. ¿Erradica la prostitución y la trata al cien por cien? No, y en muchos casos la esconde, pero hay menos explotación.

### Cifras de Clientes de Prostitución en España

¿En qué cifras nos movemos en España de clientes de la prostitución? ¿Es el país con más clientes de Europa?

Hay una encuesta que dice, y no lo digo yo, porque a todos los que les pregunto me dicen que no, que un 38% de los hombres españoles reconoce utilizarla, lo que corresponde a 8 millones y medio. Es el país de Europa que más consume y uno de los más del mundo, porque la prostitución está muy normalizada y legitimada.

### Razones de la Normalización

¿A qué puede deberse esto?

A que está enquistada desde siempre, de toda la vida, y se normaliza también por la falta de educación sexual. **No la hay. Nuestros chavales están 'educando' la sexualidad en la pornografía, lo que les hace creer que tienen derecho a comprar a un ser humano.** Nos hace falta muchísima educación en los entornos familiares y en los colegios. La Ley Celáa, de la que todo el mundo habla pestes, aborda la sexualidad como una asignatura, lo cual es magnífico.

### Futuro de la Juventud y la Prostitución

¿Cómo se enfrentan los jóvenes a la prostitución, es decir, cómo viene el futuro?

Viene lleno de pornografía, lo que llamo prostitución 2.0, la pornografía en vivo. Para escribir mi último libro, 'PornoXplotación', investigué durante casi cinco años junto a un inspector de la policía y vimos que la prostitución está migrando a esas fronteras digitales, y es a eso a lo que vamos a tener que enfrentarnos. La inteligencia artificial, que es magnífica y nos facilita la vida, también va a dejar muchas víctimas en los dos lados: por una parte niñas explotadas, secuestradas, engañadas, grooming, sexting..., y al otro lado de la pantalla niños cada vez más jóvenes que llegan a la pornografía sin un pensamiento crítico. Es un fenómeno que parece inocuo pero es peligrosísimo, turbio y en muchos casos a quienes la consumen desde muy temprana edad los vemos comprando mujeres en las rotondas a los 18, 19, 20 años... **Es algo que no debería tener cabida en este siglo.**

### Solución Propuesta

¿Cuál es tu solución a esta lacra?

No es una lacra, sino un fenómeno delictivo, porque trata y prostitución van de la mano. Porque no se puede hablar de prostitución sin hablar de trata sexual, ni de esta sin la primera, y no se puede hablar de pornografía sin hablar de prostitución. Hay dos pilares muy importantes: la legislación -leyes orgánicas que penalicen este fenómeno delictivo, a los proxenetas y a los demandantes de sexo de pago como cómplices, y que den derechos a las mujeres- y la educación sexual de los jóvenes. A partir de ahí intentaremos poner un grano de arena todos, madres y padres, como educadores, porque si no lo dejamos todo en el tejado de los políticos. ¿Qué puedo hacer yo en mi casa? Soy educador. ¿Qué hago yo? Soy cineasta y utilizo el cine como herramienta de transformación social, acabo de ganar un Goya con 'Biografía del cadáver de una mujer' y esto ha supuesto un altavoz de gran impacto, veo cómo se empieza a mover ficha. Hasta ahora no ha habido una voluntad política ni en el PP ni en el PSOE, pero no es el momento de parchear, sino de hacer leyes con valentía.

In [22]:
# display(Markdown(resultado.raw))